In [1]:
import numpy as np
import cvxopt
from cvxopt import matrix, solvers, spmatrix, sparse
import scipy
from scipy.sparse import csr_matrix, hstack, vstack
import pickle
import gc
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MaxAbsScaler

In [2]:
def mape_score(y_true, y_predicted):
    return np.mean(np.abs(y_true - y_predicted) / y_true)

In [3]:
y = np.load('train_test_split/y_train.npy')
print(y.shape)
s = np.load('train_test_split/s_train.npy')
print(s.shape)
loader = np.load('train_test_split/X_train.npz')
X = csr_matrix((loader['data'], loader['indices'], loader['indptr']), shape=loader['shape'])
shape = loader['shape']
print(shape)

D = 19
N = X.shape[1]
P = int(X.shape[0] / D)

(171684,)
(9036,)
[171684  23379]


In [4]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            62G        5.2G         31G        1.7G         26G         55G
Swap:            0B          0B          0B


In [5]:
cut = 800
if cut != 0:
    X = X[:D*cut, :]
    y = y[:D*cut]
    s = s[:cut]
P = int(X.shape[0] / D)

In [6]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            62G        1.2G         35G        1.7G         26G         59G
Swap:            0B          0B          0B


In [7]:
scaler = MaxAbsScaler(copy=False)
X = scaler.fit_transform(X)

In [8]:
X.nnz / X.shape[0] / X.shape[1]

0.09662130994752376

**======================================================================================**

In [9]:
results = {
    'mse_train': {}, 'mse_test': {},
    'mape_train': {}, 'mape_test': {},
    'clf_train': {}, 'clf_test': {}, 
    'native3_train': {}, 'native3_test': {},
    'lambda_opt_max': {}
}

In [10]:
P_opt = cvxopt.sparse(matrix(np.load('P_opt_dense.npy')[:P*D, :P*D]))
q_opt = -np.ones(P*D)
q_opt = matrix(q_opt)
eq_constraintA = cvxopt.sparse(matrix(np.diag(y)))
eq_constraintb = matrix(np.zeros(P*D))
G_opt = cvxopt.sparse(matrix(np.vstack([-np.eye(P*D), np.eye(P*D)])))

In [11]:
native_indices = np.arange(P) * D
native_X = X[native_indices].toarray()
native_X.shape
X_T_X = native_X.T @ native_X

In [12]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            62G        9.1G         27G        1.7G         26G         51G
Swap:            0B          0B          0B


In [19]:
y_test = np.load('train_test_split/y_train.npy')[-cut * D:]
print(y_test.shape)
s_test = np.load('train_test_split/s_train.npy')[-cut:]
print(s_test.shape)
loader = np.load('train_test_split/X_train.npz')
X_test = csr_matrix((loader['data'], loader['indices'], 
                     loader['indptr']), shape=loader['shape'])[-cut * D:, :]
print(X_test.shape)
X_test = scaler.transform(X_test)

(15200,)
(800,)
(15200, 23379)


In [14]:
G_opt = cvxopt.sparse(matrix(-np.eye(P*D)))
h_opt = matrix(np.zeros(P*D))

In [20]:
%%time
for C_r in np.power(10.0, np.arange(-1, 3)):
    print('C_r:', C_r)
    C_r_str = f'{C_r}'
    if C_r >= 1:
        C_r_str = f'{int(C_r)}'
    suffix = f'repeat'
    a_inv_filename = f'A_inv_{cut}_{C_r_str}_{suffix}.npy'
    a_inv_ckpt = Path(a_inv_filename)
    if a_inv_ckpt.exists():
        A_inv = np.load(a_inv_filename)
    else:
        eigvals, v = scipy.linalg.eigh(scipy.sparse.eye(N) + 2*C_r*X_T_X) #coeff
        A = v @ np.diag(np.sqrt(eigvals)) @ scipy.linalg.inv(v)
        A_inv = scipy.linalg.inv(A)
        np.save(a_inv_filename, A_inv)

    b_filename = f'B_{cut}_{C_r_str}_{suffix}.npy'
    b_ckpt = Path(b_filename)
    if b_ckpt.exists():
        B = np.load(b_filename)
    else:
        B = C_r * (A_inv @ native_X.T) @ s
        np.save(b_filename, B)

    q_opt = (X @ B) * y - 1
    q_opt = matrix(q_opt)

    sol_filename = f'sol_{cut}_{C_r_str}_{suffix}.dump'
    sol_ckpt = Path(sol_filename)
    if sol_ckpt.exists():
        with open(sol_filename, 'rb') as f:
            sol = pickle.load(f)
    else:
        sol = solvers.qp(P_opt, q_opt, G_opt, h_opt, eq_constraintA, eq_constraintb,
                         options={'max_iters':50})
        with open(sol_filename, 'wb') as f:
            pickle.dump(sol, f)

    lambda_opt = np.array(sol['x'])[:, 0]
    print('max lambda:', (lambda_opt * y).sum())
    results['lambda_opt_max'][C_r_str] = lambda_opt.max()

    w_filename = f'w_{cut}_{C_r_str}_{suffix}.npy'
    w_ckpt = Path(w_filename)
    if w_ckpt.exists():
        w = np.load(w_filename)
    else:
        w_transformed = A_inv @ X.T @ (lambda_opt * y)
        w = A_inv @ (w_transformed + B)
        np.save(w_filename, w)
    print('w norm', np.linalg.norm(w))

    predicted_affinities = (X_test @ w).reshape(s_test.shape[0], D)
    predicted_affinities_train = (X @ w).reshape(s.shape[0], D)

    results['mse_test'][C_r_str] = mean_squared_error(s_test, predicted_affinities[:, 0])
    results['mape_test'][C_r_str] = mape_score(s_test, predicted_affinities[:, 0])
    results['mse_train'][C_r_str] = mean_squared_error(s, predicted_affinities_train[:, 0])
    results['mape_train'][C_r_str] = mape_score(s, predicted_affinities_train[:, 0])

    for i in range(1, D):
        predicted_affinities[:, i] = (predicted_affinities[:, i] > predicted_affinities[:, 0])
        predicted_affinities_train[:, i] = (predicted_affinities_train[:, i] 
                                            > predicted_affinities_train[:, 0])
    results['clf_test'][C_r_str] = sum(predicted_affinities.sum(1) >= D - 1) / s_test.shape[0]
    results['native3_test'][C_r_str] = sum(predicted_affinities.sum(1) >= D - 3) / s_test.shape[0]
    results['clf_train'][C_r_str] = sum(predicted_affinities_train.sum(1) >= D - 1) / s.shape[0]
    results['native3_train'][C_r_str] = sum(predicted_affinities_train.sum(1) >= D - 3) / s.shape[0]

C_r: 0.1
max lambda: -1.3201202356226086e-22
w norm 4.416065773562491
C_r: 1.0
max lambda: -8.676753937151074e-22
w norm 5.059993036995107
C_r: 10.0
max lambda: -1.6570981737696922e-21
w norm 5.179942278732299
C_r: 100.0
max lambda: -6.34678689594435e-21
w norm 5.194690552908752
CPU times: user 15.5 s, sys: 12.6 s, total: 28.1 s
Wall time: 2min 23s


In [ ]:
%%time
for C in [1, 100, 10000]:
    h_opt = matrix(np.hstack([np.zeros(P*D), C*np.ones(P*D)]))
    for C_r in np.power(10.0, np.arange(-1, 2)):
        print('C': C, 'C_r:', C_r)
        C_r_str = f'{C_r}'
        if C_r >= 1:
            C_r_str = f'{int(C_r)}'
        suffix = f'C{C}'
        a_inv_filename = f'A_inv_{cut}_{C_r_str}_{suffix}.npy'
        a_inv_ckpt = Path(a_inv_filename)
        if a_inv_ckpt.exists():
            A_inv = np.load(a_inv_filename)
        else:
            eigvals, v = scipy.linalg.eigh(scipy.sparse.eye(N) + 2*C_r*X_T_X) #coeff
            A = v @ np.diag(np.sqrt(eigvals)) @ scipy.linalg.inv(v)
            A_inv = scipy.linalg.inv(A)
            np.save(a_inv_filename, A_inv)

        b_filename = f'B_{cut}_{C_r_str}_{suffix}.npy'
        b_ckpt = Path(b_filename)
        if b_ckpt.exists():
            B = np.load(b_filename)
        else:
            B = C_r * (A_inv @ native_X.T) @ s
            np.save(b_filename, B)

        q_opt = (X @ B) * y - 1
        q_opt = matrix(q_opt)

        sol_filename = f'sol_{cut}_{C_r_str}_{suffix}.dump'
        sol_ckpt = Path(sol_filename)
        if sol_ckpt.exists():
            with open(sol_filename, 'rb') as f:
                sol = pickle.load(f)
        else:
            sol = solvers.qp(P_opt, q_opt, G_opt, h_opt, eq_constraintA, eq_constraintb, 
                       kktsolver='ldl', options={'kktreg':1e-9, 'abstol':1e-4, 'max_iters':50})
            with open(sol_filename, 'wb') as f:
                pickle.dump(sol, f)

        lambda_opt = np.array(sol['x'])[:, 0]
        print((lambda_opt * y).sum())
        results['lambda_opt_max'][C_r_str] = lambda_opt.max()

        w_filename = f'w_{cut}_{C_r_str}_{suffix}.npy'
        w_ckpt = Path(w_filename)
        if w_ckpt.exists():
            w = np.load(w_filename)
        else:
            w_transformed = A_inv @ X.T @ (lambda_opt * y)
            w = A_inv @ (w_transformed + B)
            np.save(w_filename, w)

#         predicted_affinities = (X_test @ w).reshape(s_test.shape[0], D)
#         predicted_affinities_train = (X @ w).reshape(s.shape[0], D)

#         results['mse_test'][C_r_str] = mean_squared_error(s_test, predicted_affinities[:, 0])
#         results['mape_test'][C_r_str] = mape_score(s_test, predicted_affinities[:, 0])
#         results['mse_train'][C_r_str] = mean_squared_error(s, predicted_affinities_train[:, 0])
#         results['mape_train'][C_r_str] = mape_score(s, predicted_affinities_train[:, 0])

#         for i in range(D):
#             predicted_affinities[:, i] = (predicted_affinities[:, i] > predicted_affinities[:, 0])
#             predicted_affinities_train[:, i] = (predicted_affinities_train[:, i] 
#                                                 > predicted_affinities_train[:, 0])
#         results['clf_test'][C_r_str] = sum(predicted_affinities.sum(1) >= D - 1) / s_test.shape[0]
#         results['native3_test'][C_r_str] = sum(predicted_affinities.sum(1) >= D - 3) / s_test.shape[0]
#         results['clf_train'][C_r_str] = sum(predicted_affinities_train.sum(1) >= D - 1) / s.shape[0]
#         results['native3_train'][C_r_str] = sum(predicted_affinities_train.sum(1) >= D - 3) / s.shape[0]

In [11]:
for C in np.power(10.0, np.arange(4, 5)):
    h_opt = matrix(np.hstack([np.zeros(P*D), C*np.ones(P*D)]))
    sol = solvers.qp(P_opt, q_opt, G_opt, h_opt,
                     eq_constraintA, eq_constraintb,
                     options={'abstol':1e-4, 'max_iters':50}) 
    C_r_str = f'{C}'
    lambda_opt = np.array(sol['x'])[:, 0]
    results['lambda_opt_max'][C_r_str] = lambda_opt.max()
    print((lambda_opt * y).sum())
    w = X.T @ (lambda_opt * y)

    predicted_affinities = (X_test @ w).reshape(s_test.shape[0], D)
    predicted_affinities_train = (X @ w).reshape(s.shape[0], D)

    results['mse_test'][C_r_str] = mean_squared_error(s_test, predicted_affinities[:, 0])
    results['mape_test'][C_r_str] = mape_score(s_test, predicted_affinities[:, 0])
    results['mse_train'][C_r_str] = mean_squared_error(s, predicted_affinities_train[:, 0])
    results['mape_train'][C_r_str] = mape_score(s, predicted_affinities_train[:, 0])

    for i in range(D):
        predicted_affinities[:, i] = (predicted_affinities[:, i] > predicted_affinities[:, 0])
        predicted_affinities_train[:, i] = (predicted_affinities_train[:, i] 
                                            > predicted_affinities_train[:, 0])
    results['clf_test'][C_r_str] = sum(predicted_affinities.sum(1) >= D - 1) / s_test.shape[0]
    results['native3_test'][C_r_str] = sum(predicted_affinities.sum(1) >= D - 3) / s_test.shape[0]
    results['clf_train'][C_r_str] = sum(predicted_affinities_train.sum(1) >= D - 1) / s.shape[0]
    results['native3_train'][C_r_str] = sum(predicted_affinities_train.sum(1) >= D - 3) / s.shape[0]

     pcost       dcost       gap    pres   dres
 0:  7.4983e-10 -1.9000e+08  4e+08  1e-04  2e-10
 1:  7.4981e-12 -1.9000e+06  4e+06  1e-06  3e-12
 2:  7.4981e-14 -1.9000e+04  4e+04  1e-08  2e-12
 3:  7.4981e-16 -1.9000e+02  4e+02  1e-10  2e-12
 4:  7.4981e-18 -1.9000e+00  4e+00  1e-12  2e-12
 5:  7.4981e-20 -1.9000e-02  4e-02  1e-14  2e-12
 6:  7.4981e-22 -1.9000e-04  4e-04  2e-16  2e-12
 7:  7.4981e-24 -1.9000e-06  4e-06  2e-16  2e-12
 8:  7.4981e-26 -1.9000e-08  4e-08  2e-16  2e-12
 9:  7.4981e-28 -1.9000e-10  4e-10  2e-16  2e-12
10:  7.4981e-30 -1.9000e-12  4e-12  1e-16  2e-12
Optimal solution found.
1.882005026387209e-29


**========================================================================================**